**워드 임베딩

* 한 단어를 길이가 비교적 짧은 밀집 벡터로 나타낸다.
* 그런데 이 밀집 벡터는 단어가 갖는 의미나 단어 간의 관계 등을 어떤 식으로든 내포하고 있다.

**Word2Vec**

    * 인접한 단어의 유사도만 계산하지, 단어간 유사도는 계산하지 못한다.
    * hidden layer
    * OOV 문제 : 단어장에 없는 단어 처리 못함
    
    word2vec_tensorflow 
    
    1) 유사도 제거
    2) n-gram 이웃 
    3) input : one-hot-encoding  -> hidden layer (word-vector) -> 
    
1. CBoW(Continuous Bag of words)
     - 주변 단어(context word)들로 중심 단어(center word)를 예측 
     - sliding window
     -  CBoW는 주변에 있는 단어들을 통해 중간에 있는 단어들을 예측하는 방법입니다.
     - CBoW는 Out-of-vocabulary(단어 집합에 없는 단어), Polysemy(다의어), 문맥 파악의 한계 등이 있습니다
         
     - CBoW 신경망 구조(얕은 신경망)에서 
         + 입력층 : 주변 단어 각각의 원-핫 벡터
         + 은닉층 : 1개의 은닉층, word-vector로 학습되는 N차원 벡터
                    활성화 함수가 없어서 투사층(projection layer)라고도 함
         + 출력층 : 중심 단어의 원-핫 벡터

2. Skip-gram
    - 중심 단어로 주변 단어를 예측

3. SGNS(Skip-Gram with Negative Sampling)
    - 네거티브 샘플링은 연산량을 줄이기 위해서 소프트맥스 함수를 사용한 
V개 중 1개를 고르는 다중 클래스 분류 문제 를 시그모이드 함수를 사용한 이진 분류 문제 로 바꾸기로 합니다
    - 입력값 : 중심단어 + 주변 단어
    - 출력값 : 이웃관계면 1
    - 단어장에서 임의 단어를 샘플링해서 주변단어(output)의 label=0(거짓)으로 생성
    
    - 모델이 실제 문맥 단어(positive)는 맞추도록 학습하고,무관한 단어들(negative samples)은 틀리도록 학습하는 방법

In [1]:
from nltk.corpus import abc
corpus = abc.sents()

print(corpus[:3])
print('코퍼스의 크기 :',len(corpus))

from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
# vector size = 학습 후 임베딩 벡터의 차원
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않아요.)
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBoW, 1은 Skip-gram.

print("모델 학습 완료!")
model_result = model.wv.most_similar("man")
print(model_result)

from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/word_embedding/w2v') 
print("모델  save 완료!")

loaded_model = KeyedVectors.load_word2vec_format("~/aiffel/word_embedding/w2v")
print("모델  load 완료!")

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]
코퍼스의 크기 : 29059
모델 학습 완료!
[('woman', 0.9233373999595642), ('skull', 0.911032497882843), ('Bang', 0.9056490063667297), ('asteroid', 0.9051957130432129), ('third', 0.9020178318023682), ('baby', 0.8993921279907227), ('dog', 0.8985978364944458), ('

In [ ]:
# Embedding projector
# python -m gensim.scripts.word2vec2tensor --input ~/aiffel/word_embedding/w2v --output ~/aiffel/word_embedding/w2v

**FastText**

- word2vec -> 문자 단위 n-gram(character-level n-gram) 표현을 학습
   ```
   n = 3인 경우
   <pa, par, art, rti, tia, ial, al>,   <partial>
   ```
- 네거티브 샘플링 
- Word2Vec과 달리 OOV와 오타에 강건하다(robust) 
  단어장에 없는 단어라도, 해당 단어의 n-gram이 다른 단어에 존재하면 이로부터 벡터값을 얻는다는 원리
  
- 한국어에서는 자소단위 FastText가 빛을 발한다

In [ ]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

**GloVe**

- 단어의 빈도를 수치화한 방법인 DTM
- 카운트 기반 :  DTM을 차원 축소하여 밀집 표현(dense representation)으로 임베딩 하는 방법이 LSA(Latent Semantic Analysis)
- 예측 기반 : Word2Vec의 경우에는 LSA보다 단어 벡터 간 유사도를 구하는 능력은 뛰어나지만, LSA처럼 코퍼스의 전체적인 통계 정보를 활용하지는 못한다는 점을 한계
- 동시 등장 행렬을 사용하고 있으니 코퍼스의 전체적인 통계 정보를 활용하는 '카운트 기반'의 방법론이면서, 손실 함수를 통해 모델을 학습시키므로 '예측 기반'의 방법론

- 전체 코퍼스에서의 동시 등장 빈도의 로그값과 중심 단어 벡터와 주변 단어 벡터의 내적값의 차이가 최소화되도록 두 벡터의 값을 학습하는 것